In [ ]:
#r "nuget: FParsec"

open FParsec

type PacketItem = | Value of int | Packet of PacketItem list

let pnumber = pint32 |>> Value
let pitem, pitemRef = createParserForwardedToRef<PacketItem, unit>()
pitemRef.Value <- pstring "[" >>. (sepBy (choice [ pnumber; pitem ]) (pstring ",")) .>> pstring "]" |>> Packet

let ppair = tuple2 (pitem .>> newline) (pitem .>> newline)

let ppairs = sepBy ppair newline

let parsePairs str =
    match run ppairs str with
    | Success(result, _, _) -> result
    | Failure(errorMsg, _, _) -> failwith errorMsg


In [ ]:
#r "nuget: FsUnit"
open FsUnitTyped

let test p str =
    match run p str with
    | Success(result, _, _)   -> Result.Ok result
    | Failure(errorMsg, _, _) -> Result.Error errorMsg

test pitem "[1]" |> shouldEqual (Result.Ok (Packet [Value 1]))
test pitem "[1,2]" |> shouldEqual (Result.Ok (Packet [Value 1; Value 2]))
test pitem "[1,2,[3,4]]" |> shouldEqual (Result.Ok (Packet [Value 1; Value 2; Packet [ Value 3; Value 4]]))

test ppair "[1,2]\n[3,4]\n" |> shouldEqual (Result.Ok (Packet [Value 1; Value 2], Packet [Value 3; Value 4]))
test ppairs "[1,2]\n[3,4]\n\n[5,6]\n[7,8]\n" |> shouldEqual (Result.Ok [(Packet [Value 1; Value 2], Packet [Value 3; Value 4]); (Packet [Value 5; Value 6], Packet [Value 7; Value 8])])

In [ ]:
//Comparisons
type ComparisonResult = | LeftSmaller | Draw | RightSmaller
let rec compare x y =
    match x,y with
    | Value _, Packet _ ->
        compare (Packet [x]) y
    | Packet _, Value _ ->
        compare x (Packet [y])
    | Value v1, Value v2 -> 
        if v1 = v2 then
            Draw
        elif v1 < v2 then
            LeftSmaller
        else 
            RightSmaller
    | Packet (j::js), Packet (k::ks) ->
        match compare j k with
        | Draw -> compare (Packet js) (Packet ks)
        | c -> c
    | Packet [], Packet [] ->
        Draw
    | Packet [], Packet _ ->
        LeftSmaller
    | Packet _, Packet [] ->
        RightSmaller


In [ ]:
let testData =
    """[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
"""

let testPairs = testData |> parsePairs

testPairs |> List.item 0 ||> compare |> shouldEqual LeftSmaller
testPairs |> List.item 1 ||> compare |> shouldEqual LeftSmaller
testPairs |> List.item 2 ||> compare |> shouldEqual RightSmaller
testPairs |> List.item 3 ||> compare |> shouldEqual LeftSmaller
testPairs |> List.item 4 ||> compare |> shouldEqual RightSmaller
testPairs |> List.item 5 ||> compare |> shouldEqual LeftSmaller
testPairs |> List.item 6 ||> compare |> shouldEqual RightSmaller
testPairs |> List.item 7 ||> compare |> shouldEqual RightSmaller

In [ ]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_13.txt")
let packets = 
    File.ReadAllText(sourcePath)

let pairs = packets |> parsePairs

let correctIndexes =
    pairs
    |> List.mapi (fun i p -> (i+1, p ||> compare))
    |> List.filter (fun (i,c) -> c = LeftSmaller)

let sum = correctIndexes |> List.sumBy fstopen System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_13.txt")
let packets = 
    File.ReadAllText(sourcePath)

let pairs = packets |> parsePairs

let correctIndexes =
    pairs
    |> List.mapi (fun i p -> (i+1, p ||> compare))
    |> List.filter (fun (i,c) -> c = LeftSmaller)

let sum = correctIndexes |> List.sumBy fst

In [ ]:
printfn "Sum of correct indexes %d" sum

## Part 2

In [ ]:
let comparator x y = 
    match compare x y with
    | Draw -> 0
    | LeftSmaller -> -1
    | RightSmaller -> +1

let divider1 = Packet [ Packet [ Value 2 ] ]
let divider2 = Packet [ Packet [ Value 6 ] ]

let sortWithDividers pairs =
    pairs 
    |> List.collect (fun (p1,p2) -> [p1; p2])
    |> List.append [divider1; divider2]
    |> List.sortWith comparator

let dividerPositions sorted =
    let p1 = sorted |> List.findIndex ((=) divider1)
    let p2 = sorted |> List.findIndex ((=) divider2)
    (p1 + 1, p2 + 1)

In [ ]:
let sorted = sortWithDividers testPairs
dividerPositions sorted |> shouldEqual (10, 14)

In [ ]:
let decoderKey = 
    pairs
    |> sortWithDividers
    |> dividerPositions 
    |> (fun (a,b) -> a * b)

printfn "Decoder key %d" decoderKey